In [1]:
from IPython import parallel as p
pc = p.Client()
pc.block = True # make results return synchronously

In [2]:
pc.ids

[0, 1, 2, 3]

In [3]:
view = pc.load_balanced_view()

In [4]:
%%px 

# run entire cell on all engines (set up a consistent environment)

# import modules onto each engine
import numpy as np
import pandas as pd

In [5]:
# import global modules

import pandas as pd
import numpy as np
import glob

dates = pd.read_pickle('./timecode.pcl')
wyears = np.load('./wyears.npy')

In [6]:
def rainrate(fr):
    lat = float(fr.split('_')[-2]) # extract latitude from file name
    lon = float(fr.split('_')[-1]) # extract longitude from file name
    
    forcing_columns = ['P','Tmax','Tmin','W']
    
    dat = pd.read_table(fr, sep=' ', names=forcing_columns)
    
    dat['coef'] = (0.-dat.Tmin)/(np.abs(dat.Tmax-dat.Tmin))
    
    dat.loc[dat.coef<0,'coef'] = 0.
    dat.loc[dat.coef>1,'coef'] = 1.
    
    dat['rain'] = dat['P']*(1-dat['coef'])
    
    rain= dat.loc[dat.rain>0,'rain'].mean()
    
    return lat,lon,rain
    

In [7]:
files = pd.read_pickle('./forcing_fluxes_filenames_lat_lon_index.df') # bring in the data frame of file paths

In [13]:
res = view.map(rainrate,files.forcing)

In [15]:
lat,lon,rr = zip(*res)

In [16]:
np.savez_compressed('./data/rainfall_rate.npz',lat=lat,lon=lon,rr=rr)

In [12]:
lat = list(lat)
lon = list(lon)
Sf = list(Sf)

arrays = [lat,lon]
tuples = list(zip(*arrays))

index = pd.MultiIndex.from_tuples(tuples, names=['latitude','longitude'])

cols = ['Sf2']

data2 = pd.DataFrame({'Sf2':Sf}, index=index)

In [13]:
data = pd.read_pickle('./budykodata.pcl')

In [14]:
data = data.append(data2)

In [15]:
data.head()

ET         P PETpenmanvic Sf  Sf2
latitude longitude                                    
14.65625 -92.21875  NaN  5.006493          NaN  0  NaN
         -92.28125  NaN  4.951291          NaN  0  NaN
14.71875 -92.21875  NaN   5.00617          NaN  0  NaN
         -92.28125  NaN  4.934073          NaN  0  NaN
         -92.34375  NaN  5.069163          NaN  0  NaN

In [16]:
del data['Sf']

In [17]:
data.head()

ET         P PETpenmanvic  Sf2
latitude longitude                                 
14.65625 -92.21875  NaN  5.006493          NaN  NaN
         -92.28125  NaN  4.951291          NaN  NaN
14.71875 -92.21875  NaN   5.00617          NaN  NaN
         -92.28125  NaN  4.934073          NaN  NaN
         -92.34375  NaN  5.069163          NaN  NaN

In [19]:
data.columns = ['ET','P','PETpenmanvic','Sf']

In [20]:
data.head()

ET         P PETpenmanvic  Sf
latitude longitude                                
14.65625 -92.21875  NaN  5.006493          NaN NaN
         -92.28125  NaN  4.951291          NaN NaN
14.71875 -92.21875  NaN   5.00617          NaN NaN
         -92.28125  NaN  4.934073          NaN NaN
         -92.34375  NaN  5.069163          NaN NaN

In [21]:
data.to_pickle('./budykodata_newSf.pcl')